In [1]:
import os
import scipy.io
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
path_dir_pub = './Public Testing Dataset/test_mat_data_public/'
path_dir_pri = './Private Testing Dataset/test_mat_data_private/'
file_path_pub = os.listdir(path_dir_pub)
file_path_pri = os.listdir(path_dir_pri)

In [3]:
datalist_pub = pd.read_csv('./Public Testing Dataset/test_datalist_public.csv')
datalist_pub['Sex'] = datalist_pub['Sex'] - 1
datalist_pub['Age'] = datalist_pub['Age'] / 50
datalist_pub['Voice handicap index - 10'] = datalist_pub['Voice handicap index - 10'] / 40
datalist_pub = datalist_pub.fillna(0)
datalist_pri = pd.read_csv('./Private Testing Dataset/test_datalist_private.csv')
datalist_pri['Sex'] = datalist_pri['Sex'] - 1
datalist_pri['Age'] = datalist_pri['Age'] / 50
datalist_pri['Voice handicap index - 10'] = datalist_pri['Voice handicap index - 10'] / 40
datalist_pri = datalist_pri.fillna(0)

In [4]:
X = []
ID = []
T = []
for i in datalist_pub['ID']:
    features = scipy.io.loadmat(path_dir_pub + i)['features']
    text = (datalist_pub[datalist_pub['ID'] == i]).values.tolist()[0]
    ID.append(text[0])
    del text[0] # del id
    
    if features.shape[0] == 200:
        temp = (features[0:100, :] + features[100:200, :])/2
        X.append(temp)
    elif features.shape[0] == 300:
        temp = (features[0:100, :] + features[100:200, :] + features[200:300, :])/3
        X.append(temp)
    elif features.shape[0] == 150:
        temp = (features[0:100, :] + features[50:150, :])/2
        X.append(temp)
    else:
        temp = features[0:100, :]
        X.append(temp)
    T.append(text)
    
for i in datalist_pri['ID']:
    features = scipy.io.loadmat(path_dir_pri + i)['features']
    text = (datalist_pri[datalist_pri['ID'] == i]).values.tolist()[0]
    ID.append(text[0])
    del text[0] # del id
    
    if features.shape[0] == 200:
        temp = (features[0:100, :] + features[100:200, :])/2
        X.append(temp)
    elif features.shape[0] == 300:
        temp = (features[0:100, :] + features[100:200, :] + features[200:300, :])/3
        X.append(temp)
    elif features.shape[0] == 150:
        temp = (features[0:100, :] + features[50:150, :])/2
        X.append(temp)
    else:
        temp = features[0:100, :]
        X.append(temp)
    T.append(text)

print('training data done...')

training data done...


In [5]:
print(len(X))
print(len(T))
print(len(ID))

999
999
999


In [6]:
X = np.array(X, dtype='float32')
T = np.array(T, dtype='float32')

In [7]:
X_T = np.transpose(X, (0, 2, 1))

In [8]:
model_path = './fold_model/0518/'
model_file = os.listdir(model_path)
models = []
for i in model_file:
    models.append(tf.keras.models.load_model(model_path + i))
    print(i + ' append successed..')

fold10_best_model.h5 append successed..
fold1_best_model.h5 append successed..
fold2_best_model.h5 append successed..
fold3_best_model.h5 append successed..
fold4_best_model.h5 append successed..
fold5_best_model.h5 append successed..
fold6_best_model.h5 append successed..
fold7_best_model.h5 append successed..
fold8_best_model.h5 append successed..
fold9_best_model.h5 append successed..


In [9]:
# 創建一個列表來存儲每個模型的預測結果
predictions = []

# 對每個模型進行預測

for model in models:
    frame_pred = model.predict([X, T, X_T])
    predictions.append(frame_pred)

# frame_pred = model.predict([X, T, X_T])
# predictions.append(frame_pred)

# 將預測結果進行平均
# print(predictions[0])
ensemble_predictions = np.mean(predictions, axis=0)
Y_pred = ensemble_predictions.argmax(axis=1)

print('predict done...')

predict done...


In [10]:
Y_submit = Y_pred + 1
print(Y_submit)

[1 4 2 4 1 1 2 3 1 5 3 4 1 1 1 1 5 1 3 1 4 2 2 1 1 3 1 1 2 1 2 2 4 3 2 1 4
 3 4 2 2 1 1 1 3 2 2 1 3 1 1 1 1 1 2 1 1 1 3 4 1 1 1 2 3 5 1 3 3 3 1 2 1 2
 3 1 3 1 1 1 2 1 3 1 2 2 1 2 3 2 1 1 3 2 4 4 1 4 2 2 1 1 1 1 3 2 2 3 1 1 1
 2 3 1 1 1 1 2 1 1 1 2 1 2 1 3 1 2 1 3 3 1 1 3 4 1 4 3 5 2 1 1 4 3 2 2 2 1
 3 1 1 3 3 2 1 1 1 2 3 3 2 1 1 3 2 2 4 1 2 2 1 2 5 2 1 1 3 1 2 2 2 1 1 3 1
 1 1 5 1 2 1 1 2 3 1 4 1 1 4 3 1 4 2 3 2 2 2 1 1 1 4 2 4 3 3 1 4 1 2 2 1 1
 3 1 1 1 2 1 1 2 1 2 5 1 1 2 2 1 1 3 3 1 3 1 5 4 2 1 1 3 2 1 5 4 1 1 1 3 3
 1 1 1 1 1 3 1 2 1 3 1 2 1 3 1 2 3 2 2 4 2 4 1 1 2 1 2 1 2 1 2 1 3 2 1 1 1
 1 1 2 1 2 1 1 1 2 1 4 2 1 2 3 1 3 3 1 3 1 2 1 3 2 2 1 5 2 2 4 2 1 1 2 3 1
 4 1 3 1 1 1 2 1 3 1 2 1 1 1 1 3 4 2 4 3 2 1 1 1 3 2 3 1 1 2 1 2 1 4 1 2 1
 1 1 1 3 3 1 4 1 2 3 1 2 5 1 3 4 2 1 2 1 1 1 1 4 3 5 2 1 1 3 1 3 1 1 1 1 2
 2 1 1 3 3 2 1 2 1 2 2 5 4 2 1 1 3 2 2 1 2 2 1 3 1 2 1 1 1 1 1 1 1 1 1 3 3
 1 4 1 1 1 5 1 1 2 3 2 3 3 5 1 1 3 1 1 1 4 1 1 2 3 1 2 4 3 3 2 5 2 1 1 1 4
 1 1 2 4 3 5 1 2 1 1 5 2 

In [11]:
submit = pd.DataFrame({'ID': ID, 'Y': Y_submit})

In [13]:
submit.to_csv('./Public Testing Dataset/submit_ensemble.csv', index=False, header=False)